# Food Review Tone Analysis and Response
In this tutorial, we use EvaDB + ChatGPT to analyze whether a food review is negative or not. Based on the analysis, we then use EvaDB + ChatGPT again to form a polite response to address negative review. 

## Start Postgres

In [ ]:
!apt install postgresql
!service postgresql start

## Create User and Database

In [2]:
!sudo -u postgres psql -c "CREATE USER eva WITH SUPERUSER PASSWORD 'password'"
!sudo -u postgres psql -c "CREATE DATABASE evadb"

CREATE ROLE
CREATE DATABASE


## Install EvaDB

In [3]:
%pip uninstall -y evadb
%pip install --quiet "evadb[document]"

import warnings
warnings.filterwarnings("ignore")

import evadb
cursor = evadb.connect().cursor()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... don

Downloading: "http://ml.cs.tsinghua.edu.cn/~chenxi/pytorch-models/mnist-b07bb66b.pth" to /root/.cache/torch/hub/checkpoints/mnist-b07bb66b.pth
100%|██████████| 1.03M/1.03M [00:01<00:00, 870kB/s]
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


## Create Data Source in EvaDB
We use data source to connect EvaDB directly to underlying database systems like Postgres.

In [4]:
params = {
    "user": "eva",
    "password": "password",
    "host": "localhost",
    "port": "5432",
    "database": "evadb",
}
query = f"CREATE DATABASE postgres_data WITH ENGINE = 'postgres', PARAMETERS = {params};"
cursor.query(query).df()

,0
0,The database postgres_data has been successful...


## Create Review Table

In [17]:
cursor.query("""
USE postgres_data {
  DROP TABLE IF EXISTS review_table
}
""").df()

,status
0,success


In [18]:
cursor.query("""
USE postgres_data {
  CREATE TABLE review_table (name VARCHAR(10), review VARCHAR(1000))
}
""").df()

,status
0,success


## Load Review from CSV to Postgres

In [14]:
def create_food_review():
  with open("review.txt", "w") as f:
    f.write("name|review\n")
    f.write("Customer 1|I ordered fried rice but it is too salty.\n")
    f.write("Customer 2|I ordered burger. It tastes very good and the service is exceptional.\n")
    f.write("Customer 3|I ordered a takeout order, but the chicken sandwidth is missing from the order.\n")
    f.flush()

  import pathlib
  cwd = pathlib.Path().resolve()
  return cwd / "review.txt"

In [19]:
query = f"""
USE postgres_data {{
  COPY review_table(name, review) FROM '{str(create_food_review())}'
    DELIMITER '|' CSV HEADER
}}
"""
cursor.query(query).df()

,status
0,success


## Review Table Content
Now we have 3 reviews from different customers stored in the table.

In [ ]:
cursor.query("SELECT * FROM postgres_data.review_table;").df()

## Register ChatGPT Function inside EvaDB

In [10]:
!wget -nc https://raw.githubusercontent.com/georgia-tech-db/eva/master/evadb/udfs/chatgpt.py -O chatgpt.py

--2023-08-29 19:17:19--  https://raw.githubusercontent.com/georgia-tech-db/eva/master/evadb/udfs/chatgpt.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6965 (6.8K) [text/plain]
Saving to: ‘chatgpt.py’

chatgpt.py          100%[===================>]   6.80K  --.-KB/s    in 0s      

2023-08-29 19:17:19 (58.9 MB/s) - ‘chatgpt.py’ saved [6965/6965]



In [11]:
cursor.query("DROP UDF IF EXISTS ChatGPT;").df()
cursor.query("CREATE UDF ChatGPT IMPL 'chatgpt.py'").df()

,0
0,UDF ChatGPT successfully added to the database.


## Register OpenAI Token

In [12]:
import os
os.environ["OPENAI_KEY"] = "sk-..."

## Tone Analysis for All Reviews
Here, we use ChatGPT with customized prompt to summarize whether the review tone is "postive" or "negative".

In [21]:
cursor.query("""
SELECT ChatGPT(
  "Is the review positive or negative. Only reply 'positive' or 'negative'. Here are examples. The food is very bad: negative. The food is very good: postive.",
  review
)
FROM postgres_data.review_table;
""").df()

,chatgpt.response
0,negative
1,positive
2,negative


## Respond to Negative Reviews
EvaDB allows users to filter data based on function output. In this query, we construct a query to filter out "postive" reviews. We then use a second ChatGPT with customized prompt to propose a solution to address customer's negative reviews politely. 

In [22]:
response_df = cursor.query("""
SELECT ChatGPT(
  "Respond the the review with solution to address the review's concern",
  review
)
FROM postgres_data.review_table
WHERE ChatGPT(
  "Is the review positive or negative. Only reply 'positive' or 'negative'. Here are examples. The food is very bad: negative. The food is very good: postive.",
  review
) = "negative";
""").df()

for i in range(len(response_df)):
  print(">>>", response_df["chatgpt.response"][i][:200], " ... ...")
  print()

>>> Dear valued customer,

Thank you for bringing this to our attention. We apologize for the inconvenience caused by the excessive saltiness of your fried rice. We understand how important it is to have   ... ...

>>> Dear [Customer's Name],

Thank you for bringing this issue to our attention. We apologize for the inconvenience caused by the missing chicken sandwich in your takeout order. We understand how frustrat  ... ...

